In [7]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

In [8]:
%%capture
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "tyanfarm/gemma3-4b-hotels-mixed-5epochs-lr2e-4-lora64-ft", #model folder
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

In [9]:
!apt-get update
!apt-get install pciutils build-essential cmake curl libcurl4-openssl-dev -y
!git clone https://github.com/ggerganov/llama.cpp
!cmake llama.cpp -B llama.cpp/build \
    -DBUILD_SHARED_LIBS=ON -DGGML_CUDA=ON -DLLAMA_CURL=ON
!cmake --build llama.cpp/build --config Release -j 16 --clean-first --target llama-quantize llama-cli llama-gguf-split llama-mtmd-cli
!cp llama.cpp/build/bin/llama-* llama.cpp

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 2s (245 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem 

In [10]:
# model.save_pretrained_merged("tyanfarm/llama3-8b-hotel-addresses-finetuned-gguf", tokenizer, save_method = "merged_16bit",)
# model.save_pretrained_gguf("tyanfarm/gemma3-4b-hotels-mixed-ft-gguf", tokenizer, quantization_method="q8_0")

In [ ]:
model.push_to_hub_gguf(
        "tyanfarm/gemma3-4b-hotels-mixed-ft-gguf", # Change hf to your username!
        tokenizer,
        quantization_method = ["q5_k_m"],
        # quantization_method = ["q4_k_m"],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: Converting model to GGUF format...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

New Data Upload               : |          |  0.00B /  0.00B            

  ...a-3-4b-pt.F16-mmproj.gguf:   4%|3         | 33.5MB /  851MB            

Uploading config.json...


Unsloth: Successfully uploaded GGUF to https://huggingface.co/tyanfarm/gemma3-4b-hotels-mixed-ft-gguf
Unsloth: Cleaning up temporary files...


'tyanfarm/gemma3-4b-hotels-mixed-ft-gguf'

In [12]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Saigon Hotel có phòng Superior không?",
    }]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = True,
    return_tensors = "pt",
    return_dict = True,
)
outputs = model.generate(
    **inputs.to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 0.1, top_p = 0.95, top_k = 10,
)
tokenizer.batch_decode(outputs)

['<bos><start_of_turn>user\nSaigon Hotel có phòng Superior không?<end_of_turn>\n<start_of_turn>model\nChỉ có các loại phòng: Superior (~24 m²); Deluxe (~28 m²); Premier Deluxe (~32 m²); Executive Suite (~55 m²).<eos>']